In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Seed for reproducibility
np.random.seed(42)
random.seed(42)

# ==========================
# CONFIG & SPECIFIC PRODUCT DATA
# ==========================
NUM_SKUS = 500
START_DATE = "2024-01-01"
END_DATE = "2025-12-31"
REGIONS = ["North", "West", "South", "East"]

# Highly specific product naming components
FMCG_CATALOG = {
    "Beverages": {
        "types": ["Fruit Juice", "Sparkling Water", "Energy Drink", "Instant Coffee", "Green Tea"],
        "brands": ["Oasis", "Fizz", "Volt", "Aroma", "Leafy"],
        "variants": ["Sugar-Free", "Classic", "Tropical", "Premium Blend", "Diet"]
    },
    "Snacks": {
        "types": ["Potato Chips", "Whole Grain Biscuits", "Roasted Namkeen", "Dark Chocolate", "Corn Puffs"],
        "brands": ["Crunch", "BakeWell", "DesiSnax", "CocoaPure", "Puffo"],
        "variants": ["Sea Salt", "Masala", "Oatmeal", "70% Cocoa", "Cheese Burst"]
    },
    "Dairy": {
        "types": ["Fresh Milk", "Probiotic Curd", "Salted Butter", "Cheddar Cheese", "Flavoured Yogurt"],
        "brands": ["PureDairy", "HealthFit", "GoldChurn", "Alpine", "BerryGo"],
        "variants": ["Full Cream", "Low Fat", "Garlic Infused", "Organic", "Blueberry"]
    },
    "Personal Care": {
        "types": ["Anti-Dandruff Shampoo", "Herbal Soap", "Whitening Toothpaste", "Moisturizing Lotion", "Face Wash"],
        "brands": ["Glow", "NaturePure", "BrightSmile", "SoftSkin", "ClearUp"],
        "variants": ["Aloe Vera", "Charcoal", "Mint", "Vitamin E", "Tea Tree"]
    },
    "Home Care": {
        "types": ["Liquid Detergent", "Anti-Bac Dishwash", "Pine Floor Cleaner", "Toilet Gel", "Glass Cleaner"],
        "brands": ["Sparkle", "GermKill", "PineFresh", "DeepClean", "Crystal"],
        "variants": ["Lemon", "Lavender", "Ultra-Strong", "Ocean Breeze", "Streak-Free"]
    }
}

FESTIVALS = {
    "Diwali": ["2024-11-01", "2025-10-21"],
    "Holi": ["2024-03-25", "2025-03-14"],
    "Christmas/NewYear": ["2024-12-25", "2025-12-25", "2025-01-01", "2026-01-01"]
}

# ==========================
# 1. ENHANCED SKU MASTER (With Names and Costs)
# ==========================
sku_data = []
for i in range(1, NUM_SKUS + 1):
    category = random.choice(list(FMCG_CATALOG.keys()))
    details = FMCG_CATALOG[category]
    
    # Generate specific product name
    prod_type = random.choice(details["types"])
    brand = random.choice(details["brands"])
    variant = random.choice(details["variants"])
    product_name = f"{brand} {prod_type} - {variant}"
    
    sku_id = f"SKU{i:03d}"
    
    # Pricing logic: Sales Price vs Unit Cost
    selling_price = round(np.random.uniform(30, 800), 2)
    # Unit cost is usually 40% to 70% of selling price
    margin_multiplier = np.random.uniform(0.4, 0.7)
    unit_cost = round(selling_price * margin_multiplier, 2)

    sku_data.append([sku_id, category, product_name, selling_price, unit_cost])

sku_master = pd.DataFrame(sku_data, columns=["SKU_ID", "Category", "Product_Name", "Base_Price", "Unit_Cost"])

# ==========================
# 2. DATE RANGE & FLAGS
# ==========================
dates = pd.date_range(start=START_DATE, end=END_DATE, freq="D")
fest_dates = {pd.to_datetime(d) for dlist in FESTIVALS.values() for d in dlist}

# ==========================
# 3. SALES & TRANSACTION GENERATOR
# ==========================
sales_rows = []
inventory_state = {sku: np.random.randint(500, 2000) for sku in sku_master["SKU_ID"]}

for date in dates:
    # Random daily promotion selection (20% of SKUs on any given day)
    daily_promo_skus = set(sku_master["SKU_ID"].sample(frac=0.2))
    is_holiday = 1 if date in fest_dates else 0

    for index, row in sku_master.iterrows():
        sku = row["SKU_ID"]
        is_promo = 1 if sku in daily_promo_skus else 0
        
        # Base daily demand by category
        base_demand = {"Beverages": 40, "Snacks": 35, "Dairy": 50, "Personal Care": 15, "Home Care": 12}[row["Category"]]
        
        # Apply Multipliers
        demand = base_demand
        if date.weekday() >= 5: demand *= 1.3 # Weekend
        if is_promo: demand *= 1.5           # Promo
        if is_holiday: demand *= 2.2         # Festival
        
        # Random noise
        demand = int(max(0, np.random.normal(demand, demand * 0.15)))
        
        # Transactional Math
        units_sold = min(inventory_state[sku], demand)
        current_price = round(row["Base_Price"] * (0.85 if is_promo else 1.0), 2)
        
        revenue = round(units_sold * current_price, 2)
        total_cost = round(units_sold * row["Unit_Cost"], 2)
        profit = round(revenue - total_cost, 2)
        
        inventory_state[sku] -= units_sold
        
        # Restock logic (Simple daily check)
        if inventory_state[sku] < 200:
            inventory_state[sku] += 1000 

        sales_rows.append([
            date, sku, row["Product_Name"], row["Category"], 
            units_sold, current_price, revenue, total_cost, profit
        ])

# ==========================
# 4. FINAL EXPORT
# ==========================
sales_df = pd.DataFrame(sales_rows, columns=[
    "Date", "SKU_ID", "Product_Name", "Category", 
    "Units_Sold", "Unit_Price", "Revenue", "Total_Cost", "Profit"
])

sku_master.to_csv("sku_master_detailed.csv", index=False)
sales_df.to_csv("sales_transactions_detailed.csv", index=False)

print(f"✅ Generated {len(sales_df)} transactions for {NUM_SKUS} specific products.")

✅ Generated 365500 transactions for 500 specific products.


In [ ]:
import pandas as pd
from sqlalchemy import text
from database import engine

def import_csv_to_db():
    try:
        # 1. Load CSVs
        sku_df = pd.read_csv(r"C:\Users\amits\OneDrive\Desktop\new_shop\sku_master_detailed.csv")
        sales_df = pd.read_csv(r"C:\Users\amits\OneDrive\Desktop\new_shop\sales_transactions_detailed.csv")

        # 2. Map SKU Master: CSV (Left) -> Database (Right)
        sku_db = sku_df.rename(columns={
            "SKU_ID": "sku_id",
            "Category": "category",
            "Product_Name": "product_name",
            "Base_Price": "base_price",
            "Unit_Cost": "unit_cost"
        })

        # 3. Map Sales: CSV (Left) -> Database (Right)
        sales_db = sales_df.rename(columns={
            "Date": "sales_date",
            "SKU_ID": "sku_id",
            "Units_Sold": "units_sold",
            "Unit_Price": "unit_price",
            "Revenue": "total_price"
        })
        
        # Add required columns that aren't in the CSV
        sales_db["store_id"] = "INITIAL_LOAD"
        sales_db["channel"] = "Retail"
        sales_db["discount"] = 0

        # Filter for only the columns the DB table actually has
        sales_db = sales_db[["sales_date", "sku_id", "store_id", "units_sold", "unit_price", "total_price", "discount", "channel"]]

        print("Pushing data to PostgreSQL...")
        
        # Using 'append' so it works with the tables we just created
        sku_db.to_sql('sku_master', engine, if_exists='append', index=False)
        sales_db.to_sql('sales_transactions', engine, if_exists='append', index=False)

        print(f"✅ Successfully imported {len(sku_db)} products and {len(sales_db)} sales.")

    except Exception as e:
        print(f"❌ Mapping Error: {e}")

if __name__ == "__main__":
    import_csv_to_db()

Pushing data to PostgreSQL...
✅ Successfully imported 500 products and 365500 sales.


: 